In [1]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

ССылки на файлы

In [2]:
ASU_dataset = Path('data/train_dataset_train.csv')

Загрузка баового датасета

In [3]:

asu_frame = pd.read_csv(ASU_dataset)
asu_frame.head(2)

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Общежитие,Наличие_Матери,Наличие_Отца,Страна_Родители,Опекунство,Село,Иностранец,КодФакультета,СрБаллАттестата,Статус
0,72716,16019,2015,Жен,ОО,Английский язык,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,...,0.0,0,0.0,Россия,0.0,0.0,0.0,41.0,4.294,3
1,63306,14895,2010,Муж,ЦН,NaN,1992-04-18 00:00:00.000,NaN,NaN,NaN,...,0.0,1,1.0,Россия,0.0,0.0,0.0,28.0,67.000,4


Очистка значений датафрейма

In [4]:
asu_frame.loc[asu_frame['Пол'] == 'муж', 'Пол'] = 'Муж'

In [5]:
asu_frame['male'] = 0
asu_frame['female'] = 0

asu_frame.loc[asu_frame['Пол'] == 'Муж', 'male'] = 1
asu_frame.loc[asu_frame['Пол'] == 'Жен', 'female'] = 1
asu_frame['sex_summ'] = asu_frame['male'] + asu_frame['female']
asu_frame.loc[asu_frame['sex_summ'] != 1].shape[0]

7

In [6]:
#asu_frame['birth_year'] = asu_frame['Дата_Рождения'].dt.to_period('Y')
asu_frame['Дата_Рождения']
asu_frame['birth_year'] = pd.to_datetime(asu_frame['Дата_Рождения'])
asu_frame['birth_year'].dt.to_period('Y')
asu_frame['birth_year'].all()

asu_frame['entrance'] = pd.to_datetime(asu_frame['Год_Поступления'], format='%Y')
asu_frame['entrance'].sort_values()



4719    2001-01-01
10232   2003-01-01
8941    2004-01-01
9417    2005-01-01
7300    2007-01-01
           ...    
10228   2020-01-01
5126    2020-01-01
2748    2020-01-01
12961   2020-01-01
104     2212-01-01
Name: entrance, Length: 13584, dtype: datetime64[ns]

In [7]:
asu_frame['birth_year_int'] = asu_frame['Дата_Рождения'].str.slice(start=0, stop=4).astype('int64')
asu_frame['birth_year_int']

0        1998
1        1992
2        1999
3        1994
4        1995
         ... 
13579    1998
13580    1997
13581    1998
13582    1995
13583    1995
Name: birth_year_int, Length: 13584, dtype: int64

In [8]:
frame_size = asu_frame.shape
frame_size

(13584, 30)

In [9]:
asu_frame.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Пол', 'Основания',
       'Изучаемый_Язык', 'Дата_Рождения', 'Уч_Заведение', 'Где_Находится_УЗ',
       'Год_Окончания_УЗ', 'Пособие', 'Страна_ПП', 'Регион_ПП', 'Город_ПП',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'Статус', 'male', 'female', 'sex_summ', 'birth_year', 'entrance',
       'birth_year_int'],
      dtype='object')

In [10]:
asu_frame['Статус']

0        3
1        4
2        4
3        4
4        4
        ..
13579    3
13580    4
13581    4
13582    4
13583    4
Name: Статус, Length: 13584, dtype: int64

In [11]:
asu_frame.groupby('Статус')['СрБаллАттестата'].describe()

,count,mean,std,min,25%,50%,75%,max
Статус,,,,,,,,
-1,614.0,100.811304,429.151037,3.285,48.0,61.0,71.0,4800.0
3,4721.0,82.513324,344.527967,0.000,44.0,56.0,70.0,5000.0
4,8249.0,65.853001,166.979956,0.000,49.0,64.0,79.0,5000.0


In [12]:
asu_frame.groupby('Статус')['Село'].value_counts()

Статус  Село
-1      0.0      504
        1.0      110
 3      0.0     3539
        1.0     1132
 4      0.0     6358
        1.0     1855
Name: Село, dtype: int64

In [13]:
asu_frame.pivot_table(index='Пол', columns='Статус')

ID                             birth_year_int               \
Статус            -1             3             4             -1            3   
Пол                                                                            
Жен     83378.066667  71829.522481  68025.234389    1995.407407  1993.657785   
Муж     83746.534884  71596.500216  68424.950090    1996.889535  1994.087257   

                    female       male  ... Опекунство Пособие            \
Статус            4     -1  3  4   -1  ...          4      -1    3    4   
Пол                                    ...                                
Жен     1993.971434      1  1  1    0  ...   0.000183     0.0  0.0  0.0   
Муж     1993.848115      0  0  0    1  ...   0.000000     0.0  0.0  0.0   

            Село                     СрБаллАттестата                        
Статус        -1         3         4              -1          3          4  
Пол                                                                         
Жен     0.196296  0.265487  0.243099       72.675426  93.906158  65.087485  
Муж     0.165698  0.218832  0.192363      122.894696  70.760177  67.357506  

[2 rows x 51 columns]

In [14]:
graduate = asu_frame.loc[asu_frame['Статус'] == 4]['Статус'].count()
expelled = asu_frame.loc[asu_frame['Статус'] == 3]['Статус'].count()
vacation = asu_frame.loc[asu_frame['Статус'] == -1]['Статус'].count()
all_students = frame_size[0]
delta = all_students - graduate - expelled - vacation
f'Всего позиций {all_students} закончили ВУЗ {graduate}, отчислено {expelled}, каникулы {vacation}. Отклонение по сумме {delta}'

'Всего позиций 13584 закончили ВУЗ 8249, отчислено 4721, каникулы 614. Отклонение по сумме 0'

In [15]:
graduate_frame = asu_frame.loc[asu_frame['Статус'] == 4]
graduate_frame

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int
1,63306,14895,2010,Муж,ЦН,NaN,1992-04-18 00:00:00.000,NaN,NaN,NaN,...,0.0,28.0,67.000,4,1,0,1,1992-04-18,2010-01-01,1992
2,82469,20103,2017,Жен,ДН,Английский язык,1999-03-21 00:00:00.000,МБОУ Алтайская СОШ №5,"Алтайский край, Алтайский р-н, Алтайское с",2017.0,...,0.0,41.0,4.133,4,0,1,1,1999-03-21,2017-01-01,1999
3,81661,19838,2017,Жен,БН,Английский язык,1994-12-08 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,0.0,35.0,74.000,4,0,1,1,1994-12-08,2017-01-01,1994
4,81509,19803,2017,Жен,БН,Английский язык,1995-06-28 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,0.0,28.0,53.000,4,0,1,1,1995-06-28,2017-01-01,1995
6,57717,16945,2013,Муж,СН,NaN,1995-05-10 00:00:00.000,NaN,NaN,NaN,...,0.0,51.0,67.000,4,1,0,1,1995-05-10,2013-01-01,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13578,56928,17038,2013,Жен,СН,Английский язык,1989-12-21 00:00:00.000,"Автономная некоммерческая организация ВПО ""Алт...","г. Барнаул, Алтайский край",2011.0,...,0.0,26.0,51.000,4,0,1,1,1989-12-21,2013-01-01,1989
13580,71516,19433,2015,Жен,ОО,Немецкий язык,1997-09-19 00:00:00.000,"МКОУ ""Георгиевская СОШ""","Алтайский край, Ребрихинский р-н, Георгиевка с",2015.0,...,0.0,26.0,82.000,4,0,1,1,1997-09-19,2015-01-01,1997
13581,76970,18723,2016,Муж,СН,Английский язык,1998-03-20 00:00:00.000,МБОУ СОШ №113 им. Сергея Семенова,"Россия, Алтайский край, г Барнаул",2016.0,...,0.0,41.0,3.400,4,1,0,1,1998-03-20,2016-01-01,1998
13582,48548,14663,2012,Жен,ОО,NaN,1995-07-06 00:00:00.000,NaN,NaN,NaN,...,0.0,24.0,95.000,4,0,1,1,1995-07-06,2012-01-01,1995


In [16]:
def doly(x):
    
    gh = x['ID'].count() / frame_size[0]
    print(gh)
    return gh

asu_frame.groupby(['Пол', 'Статус']).apply(doly)

0.01987632508833922
0.1768256772673734
0.40201707891637223
0.025323910482921083
0.1704210836277974
0.2050206124852768


Пол  Статус
Жен  -1        0.019876
      3        0.176826
      4        0.402017
Муж  -1        0.025324
      3        0.170421
      4        0.205021
dtype: float64

In [17]:
asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()

/tmp/ipykernel_97976/3035972624.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()
/tmp/ipykernel_97976/3035972624.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()
/tmp/ipykernel_97976/3035972624.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of

,count,unique,top,freq,first,last
birth_year,,,,,,
1964,1,1,2016-01-01 00:00:00,1,2016-01-01 00:00:00,2016-01-01 00:00:00
1965,1,1,2014-01-01 00:00:00,1,2014-01-01 00:00:00,2014-01-01 00:00:00
1966,1,1,2015-01-01 00:00:00,1,2015-01-01 00:00:00,2015-01-01 00:00:00
1969,1,1,2014-01-01 00:00:00,1,2014-01-01 00:00:00,2014-01-01 00:00:00
1971,1,1,2015-01-01 00:00:00,1,2015-01-01 00:00:00,2015-01-01 00:00:00
1972,3,3,2013-01-01 00:00:00,1,2013-01-01 00:00:00,2016-01-01 00:00:00
1973,2,2,2016-01-01 00:00:00,1,2015-01-01 00:00:00,2016-01-01 00:00:00
1974,1,1,2016-01-01 00:00:00,1,2016-01-01 00:00:00,2016-01-01 00:00:00
1975,1,1,2015-01-01 00:00:00,1,2015-01-01 00:00:00,2015-01-01 00:00:00


In [18]:
old_year = datetime.datetime.strptime('1986', '%Y')
asu_frame.loc[(asu_frame['birth_year'] < old_year) & (asu_frame['СрБаллАттестата'] > 20)]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int
17,53187,12426,2012,Муж,ДН,NaN,1982-06-09 00:00:00.000,NaN,NaN,NaN,...,NaN,26.0,88.0,3,1,0,1,1982-06-09,2012-01-01,1982
21,58638,16973,2012,Жен,ОО,NaN,1976-09-20 00:00:00.000,NaN,NaN,NaN,...,0.0,48.0,63.0,4,0,1,1,1976-09-20,2012-01-01,1976
47,56984,12412,2013,Муж,ДН,Английский язык,1980-11-07 00:00:00.000,Алтайский государственный университет,"г.Барнаул, Алтайский край",2003.0,...,NaN,26.0,80.0,3,1,0,1,1980-11-07,2013-01-01,1980
52,67548,15633,2014,Жен,ОО,Английский язык,1967-02-07 00:00:00.000,"ФГБОУ ВПО ""Алтайский государственный университет""","Алтайский край, Барнаул г",1992.0,...,0.0,31.0,80.0,3,0,1,1,1967-02-07,2014-01-01,1967
53,59684,12812,2013,Муж,ДН,Английский язык,1985-11-19 00:00:00.000,"ГОУ СПО СО ""Каменск-Уральский техникум торговл...",г. Каменск-Уральский,2010.0,...,NaN,45.0,85.0,3,1,0,1,1985-11-19,2013-01-01,1985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13539,80890,21005,2017,Жен,ОО,Английский язык,1976-09-22 00:00:00.000,Всероссийский заочный финансово-экономический ...,"Москва г,",1998.0,...,0.0,27.0,39.0,3,0,1,1,1976-09-22,2017-01-01,1976
13556,65615,15431,2014,Жен,ОО,Немецкий язык,1970-08-25 00:00:00.000,"НГУ ""Современная гуманитарная академия""","Московская обл, Москва г",2006.0,...,0.0,34.0,95.0,4,0,1,1,1970-08-25,2014-01-01,1970
13559,65958,18141,2014,Жен,ОО,Английский язык,1985-07-21 00:00:00.000,"МБОУ ""Сибирская СОШ""","Алтайский край, Первомайский р-н, Сибирский п",2002.0,...,0.0,25.0,88.0,3,0,1,1,1985-07-21,2014-01-01,1985
13566,56942,14815,2013,Жен,ОО,Английский язык,1980-07-06 00:00:00.000,"АНО ВПО ""Алтайская академия экономики и права ...",Алтайский край г. Барнаул,2002.0,...,NaN,26.0,84.0,4,0,1,1,1980-07-06,2013-01-01,1980


In [19]:
old_year = datetime.datetime.strptime('2004', "%Y")


In [20]:
asu_frame.groupby('Код_группы')['ID'].count().max()

23

# Очистка первичного фрейма

In [21]:
clean_frame = pd.DataFrame()

In [22]:
clean_frame = asu_frame.copy()

Колонка иностранный язык

In [23]:
clean_frame.loc[clean_frame['Изучаемый_Язык'].isnull(), 'Изучаемый_Язык'] = '-'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Англиийский', 'Изучаемый_Язык'] = 'Английский язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Иностранный язык (Английский)', 'Изучаемый_Язык'] = 'Английский язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Иностранный язык (Немецкий)', 'Изучаемый_Язык'] = 'Немецкий язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Русский язык', 'Изучаемый_Язык'] = '-'

In [24]:
clean_frame.loc[clean_frame['Пол'].isnull(), 'Пол'] = '-'

In [25]:
clean_frame.loc[clean_frame['Уч_Заведение'].isnull(), 'Уч_Заведение'] = '-'

In [26]:
clean_frame.loc[clean_frame['Где_Находится_УЗ'].isnull(), 'Где_Находится_УЗ'] = '-'

In [27]:
clean_frame.loc[clean_frame['Пособие'].isnull()]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int
11,52217,16851,2012,Жен,ОО,Английский язык,1991-04-14 00:00:00.000,-,-,NaN,...,NaN,24.0,55.0,4,0,1,1,1991-04-14,2012-01-01,1991
17,53187,12426,2012,Муж,ДН,-,1982-06-09 00:00:00.000,-,-,NaN,...,NaN,26.0,88.0,3,1,0,1,1982-06-09,2012-01-01,1982
19,65422,15423,2014,Жен,ОО,Английский язык,1990-07-30 00:00:00.000,"ФГБОУ ВПО ""Алтайский государственный университет""","Алтайский край, Барнаул г",2012.0,...,0.0,51.0,56.0,3,0,1,1,1990-07-30,2014-01-01,1990
30,56807,15640,2013,Муж,СН,Немецкий язык,1988-01-21 00:00:00.000,"ФГВООУ ВПО ""Военный учебно-научный центр Сухо...",г. Москва,2012.0,...,NaN,45.0,65.0,3,1,0,1,1988-01-21,2013-01-01,1988
47,56984,12412,2013,Муж,ДН,Английский язык,1980-11-07 00:00:00.000,Алтайский государственный университет,"г.Барнаул, Алтайский край",2003.0,...,NaN,26.0,80.0,3,1,0,1,1980-11-07,2013-01-01,1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13562,72563,17444,2015,Жен,СН,Немецкий язык,1994-07-18 00:00:00.000,"КГБПОУ ""Алтайская академия гостеприимства""","Алтайский край, Барнаул г",2015.0,...,0.0,24.0,64.0,3,0,1,1,1994-07-18,2015-01-01,1994
13566,56942,14815,2013,Жен,ОО,Английский язык,1980-07-06 00:00:00.000,"АНО ВПО ""Алтайская академия экономики и права ...",Алтайский край г. Барнаул,2002.0,...,NaN,26.0,84.0,4,0,1,1,1980-07-06,2013-01-01,1980
13572,71911,17229,2015,Жен,СН,Немецкий язык,1989-07-09 00:00:00.000,"ФГБОУ ВПО ""Алтайский государственный университет""","Алтайский край, Барнаул г",2012.0,...,0.0,26.0,52.0,3,0,1,1,1989-07-09,2015-01-01,1989
13575,68066,15469,2014,Муж,СН,Английский язык,1994-02-06 00:00:00.000,"МБОУ ""СОШ №19""","Алтайский край, Новоалтайск г",2012.0,...,0.0,26.0,50.0,3,1,0,1,1994-02-06,2014-01-01,1994


In [28]:
clean_frame.loc[clean_frame['Страна_ПП'].isnull(), 'Страна_ПП'] = '-'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Кыргызия', 'Страна_ПП'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Киргизия', 'Страна_ПП'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Кыргызстан', 'Страна_ПП'] = 'Кыргызская Республика'

clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан ВКО', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан Респ', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан респ', 'Страна_ПП'] = 'Республика Казахстан'

clean_frame.loc[clean_frame['Страна_ПП'] == 'Таджикистан', 'Страна_ПП'] = 'Республика Таджикистан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Таджикистан Респ', 'Страна_ПП'] = 'Республика Таджикистан'

clean_frame.loc[clean_frame['Страна_ПП'] == 'РОССИЯ', 'Страна_ПП'] = 'Россия'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Росссия', 'Страна_ПП'] = 'Россия'

In [29]:
clean_frame.loc[clean_frame['Регион_ПП'].isnull(), 'Регион_ПП'] = '-'

In [30]:
clean_frame.loc[clean_frame['Город_ПП'].isnull(), 'Город_ПП'] = '-'

In [31]:
clean_frame.loc[clean_frame['Общежитие'].isnull(), 'Общежитие'] = 0

In [32]:
clean_frame.loc[clean_frame['Страна_Родители'].isnull(), 'Страна_Родители'] = '-'

clean_frame.loc[clean_frame['Страна_Родители'] == 'КНР', 'Страна_Родители'] = 'Китай'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Казахстан', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'казахстан', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Кыргызстан', 'Страна_Родители'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Киргизия', 'Страна_Родители'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Таджикистан', 'Страна_Родители'] = 'Республика Таджикистан'


In [33]:
clean_frame.loc[clean_frame['Опекунство'].isnull()]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int


In [34]:

clean_frame.loc[(clean_frame['Село'].isnull()) & (~clean_frame['Город_ПП'].isin(['Барнаул', 'Барнаул г',])), 'Село'] = 1
clean_frame.loc[(clean_frame['Село'].isnull()) & (clean_frame['Город_ПП'].isin(['Барнаул', 'Барнаул г',])), 'Село'] = 0
clean_frame.loc[(clean_frame['Село'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int


In [35]:

clean_frame.loc[(clean_frame['Иностранец'].isnull()) & (~clean_frame['Иностранец'].isin(['Россия'])), 'Иностранец'] = 1
clean_frame.loc[(clean_frame['Иностранец'].isnull()) & (clean_frame['Иностранец'].isin(['Россия'])), 'Иностранец'] = 0
clean_frame.loc[(clean_frame['Иностранец'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int


In [36]:
clean_frame.loc[(clean_frame['КодФакультета'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int


In [37]:
clean_frame.loc[(clean_frame['КодФакультета'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int


In [38]:
clean_frame.loc[(clean_frame['СрБаллАттестата'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int


In [39]:
clean_frame.loc[(clean_frame['Статус'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int


In [40]:
clean_frame.groupby('СрБаллАттестата')['ID'].count()

СрБаллАттестата
0.000       10
3.000       16
3.066        2
3.100        2
3.111        1
            ..
4769.000     2
4800.000     1
4866.000     1
4947.000     1
5000.000     3
Name: ID, Length: 462, dtype: int64

In [41]:
asu_frame.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Пол', 'Основания',
       'Изучаемый_Язык', 'Дата_Рождения', 'Уч_Заведение', 'Где_Находится_УЗ',
       'Год_Окончания_УЗ', 'Пособие', 'Страна_ПП', 'Регион_ПП', 'Город_ПП',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'Статус', 'male', 'female', 'sex_summ', 'birth_year', 'entrance',
       'birth_year_int'],
      dtype='object')

In [42]:
columns_to_csv = ['Год_Поступления', 'Пол', 'Основания', 'Изучаемый_Язык', 'Дата_Рождения', 'Страна_ПП', 'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'Статус', 'male', 'female', 'sex_summ', 'birth_year', 'entrance']

In [43]:
clean_frame.columns[clean_frame.isna().any()].tolist()

['Год_Окончания_УЗ', 'Пособие']

In [44]:
clean_frame.loc[:, ['Основания', 'male', 'female']].head(10)

,Основания,male,female
0,ОО,0,1
1,ЦН,1,0
2,ДН,0,1
3,БН,0,1
4,БН,0,1
5,СН,1,0
6,СН,1,0
7,СН,0,1
8,СН,0,1
9,ОО,1,0


# Составление справочников кодов для текстовых колонок

## Справочник Основания

In [45]:
input_train = clean_frame.loc[:, ['Основания', 'male', 'female']].head(10).copy()

In [46]:
basis_frame = clean_frame.groupby('Основания')['ID'].count()
basis_list = basis_frame.index.to_list()
basis_dict = {}
for num, name in enumerate(basis_list):
    basis_dict[name] = num
basis_dict

{'БН': 0, 'ДН': 1, 'ОО': 2, 'СН': 3, 'ЦН': 4}

In [47]:
def get_code(x):
    return basis_dict[x]
#clean_frame['basis'] = clean_frame['Основания'].apply(get_code)
#clean_frame.head(3)

In [48]:
def get_code(x):
    return basis_dict[x]
input_train['basis'] = input_train['Основания'].apply(get_code)
input_train.head(3)

,Основания,male,female,basis
0,ОО,0,1,2
1,ЦН,1,0,4
2,ДН,0,1,1


In [49]:


scaler = MinMaxScaler()
input_train_scaled = scaler.fit_transform(input_train.loc[:, ['male', 'female', 'basis']])
input_train_scaled

array([[0.  , 1.  , 0.5 ],
       [1.  , 0.  , 1.  ],
       [0.  , 1.  , 0.25],
       [0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [1.  , 0.  , 0.75],
       [1.  , 0.  , 0.75],
       [0.  , 1.  , 0.75],
       [0.  , 1.  , 0.75],
       [1.  , 0.  , 0.5 ]])

## Справочник Изучаемый_Язык

In [50]:
input_train = clean_frame.loc[:, ['Изучаемый_Язык', 'male', 'female']].head(10).copy()
input_train

,Изучаемый_Язык,male,female
0,Английский язык,0,1
1,-,1,0
2,Английский язык,0,1
3,Английский язык,0,1
4,Английский язык,0,1
5,Английский язык,1,0
6,-,1,0
7,Английский язык,0,1
8,Английский язык,0,1
9,Английский язык,1,0


In [51]:
basis_frame = clean_frame.groupby('Изучаемый_Язык')['ID'].count()
basis_list = basis_frame.index.to_list()
basis_dict = {}
for num, name in enumerate(basis_list):
    basis_dict[name] = num
basis_dict

{'-': 0, 'Английский язык': 1, 'Немецкий язык': 2, 'Французский язык': 3}

In [52]:
def get_code(x):
    return basis_dict[x]
#clean_frame['language'] = clean_frame['Изучаемый_Язык'].apply(get_code)
#clean_frame.head(3)

In [53]:
def get_code(x):
    return basis_dict[x]
input_train['language'] = input_train['Изучаемый_Язык'].apply(get_code)
input_train.head(3)

,Изучаемый_Язык,male,female,language
0,Английский язык,0,1,1
1,-,1,0,0
2,Английский язык,0,1,1


In [54]:
scaler = MinMaxScaler()
input_train_scaled = scaler.fit_transform(input_train.loc[:, ['male', 'female', 'language']])
input_train_scaled

array([[0., 1., 1.],
       [1., 0., 0.],
       [0., 1., 1.],
       [0., 1., 1.],
       [0., 1., 1.],
       [1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 1.],
       [0., 1., 1.],
       [1., 0., 1.]])

## Цикл по справочникам

In [55]:
input_train = clean_frame.head(10).copy()
input_train

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,Статус,male,female,sex_summ,birth_year,entrance,birth_year_int
0,72716,16019,2015,Жен,ОО,Английский язык,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,...,0.0,41.0,4.294,3,0,1,1,1998-02-08,2015-01-01,1998
1,63306,14895,2010,Муж,ЦН,-,1992-04-18 00:00:00.000,-,-,NaN,...,0.0,28.0,67.000,4,1,0,1,1992-04-18,2010-01-01,1992
2,82469,20103,2017,Жен,ДН,Английский язык,1999-03-21 00:00:00.000,МБОУ Алтайская СОШ №5,"Алтайский край, Алтайский р-н, Алтайское с",2017.0,...,0.0,41.0,4.133,4,0,1,1,1999-03-21,2017-01-01,1999
3,81661,19838,2017,Жен,БН,Английский язык,1994-12-08 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,0.0,35.0,74.000,4,0,1,1,1994-12-08,2017-01-01,1994
4,81509,19803,2017,Жен,БН,Английский язык,1995-06-28 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,0.0,28.0,53.000,4,0,1,1,1995-06-28,2017-01-01,1995
5,72669,18733,2015,Муж,СН,Английский язык,1997-11-13 00:00:00.000,"МБОУ ""СОШ №110""","Алтайский край, Барнаул г",2015.0,...,0.0,41.0,3.692,3,1,0,1,1997-11-13,2015-01-01,1997
6,57717,16945,2013,Муж,СН,-,1995-05-10 00:00:00.000,-,-,NaN,...,0.0,51.0,67.000,4,1,0,1,1995-05-10,2013-01-01,1995
7,65208,18196,2014,Жен,СН,Английский язык,1996-05-20 00:00:00.000,"МБОУ ""Калманская СОШ""","Алтайский край, Калманский р-н, Калманка с",2014.0,...,0.0,51.0,58.000,3,0,1,1,1996-05-20,2014-01-01,1996
8,78608,19620,2016,Жен,СН,Английский язык,1994-09-05 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Россия, Алтайский край, г Барнаул",2016.0,...,0.0,26.0,78.000,4,0,1,1,1994-09-05,2016-01-01,1994
9,83857,19966,2017,Муж,ОО,Английский язык,1999-06-24 00:00:00.000,МБОУ Лицей №101,"Алтайский край, г. Барнаул",2017.0,...,0.0,34.0,65.000,3,1,0,1,1999-06-24,2017-01-01,1999


In [56]:
column_types = clean_frame.dtypes.to_frame()
column_types

,0
ID,int64
Код_группы,int64
Год_Поступления,int64
Пол,object
Основания,object
Изучаемый_Язык,object
Дата_Рождения,object
Уч_Заведение,object
Где_Находится_УЗ,object
Год_Окончания_УЗ,float64


In [57]:
digit_columns = ['Код_группы', 'Год_Поступления', 'Пособие', 'Общежитие',
'Наличие_Матери', 'Наличие_Отца', 'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
'Статус', 'male', 'female', 'birth_year_int']

In [58]:
# В данном коде мы извлекаем наименования тех колонок, которые содержат данные, отличные от числа (float or int)


column_types = clean_frame.dtypes.to_frame().isin(['object', 'datetime64[ns]'])
columns_need_to_code = list(column_types.loc[column_types[0] == True].to_dict()[0])
columns_need_to_code

[]

In [59]:
columns_to_code = {
    'Основания': 'basis',
    'Изучаемый_Язык': 'language',    
    'Страна_ПП': 'country',
    'Регион_ПП': 'region',
    'Город_ПП': 'city',
    'Страна_Родители': 'parents_country',
}

In [60]:
def code_dict(column_name: str) -> dict[str, int]:
    code_frame = clean_frame.groupby(column_name)['ID'].count()
    code_list = code_frame.index.to_list()
    target_dict = {}
    for num, name in enumerate(code_list):
        target_dict[name] = num
    return target_dict

In [61]:
def test(x, arg):
    mes = f'chore - {x}, change to {arg[x]}'
    print(mes)

#input_train['Изучаемый_Язык'].apply(test, args=({'Английский язык': 'o', '-': 'g'},))

In [62]:
# Функция для преобразования слова в код по заданному правилу

def get_code(x: pd.Series, code_dict: dict[str, int]) -> pd.Series:    
    return code_dict[x]



In [63]:
codes_rule = {}
for name, code in columns_to_code.items():
    target_dict = code_dict(name)
    codes_rule[name] = target_dict
    input_train[columns_to_code[name]] = input_train[name].apply(get_code, args=(target_dict,))

input_train

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,sex_summ,birth_year,entrance,birth_year_int,basis,language,country,region,city,parents_country
0,72716,16019,2015,Жен,ОО,Английский язык,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,...,1,1998-02-08,2015-01-01,1998,2,1,8,14,74,10
1,63306,14895,2010,Муж,ЦН,-,1992-04-18 00:00:00.000,-,-,NaN,...,1,1992-04-18,2010-01-01,1992,4,0,8,14,74,10
2,82469,20103,2017,Жен,ДН,Английский язык,1999-03-21 00:00:00.000,МБОУ Алтайская СОШ №5,"Алтайский край, Алтайский р-н, Алтайское с",2017.0,...,1,1999-03-21,2017-01-01,1999,1,1,8,14,31,10
3,81661,19838,2017,Жен,БН,Английский язык,1994-12-08 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,1,1994-12-08,2017-01-01,1994,0,1,8,14,1310,10
4,81509,19803,2017,Жен,БН,Английский язык,1995-06-28 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,1,1995-06-28,2017-01-01,1995,0,1,8,14,1310,10
5,72669,18733,2015,Муж,СН,Английский язык,1997-11-13 00:00:00.000,"МБОУ ""СОШ №110""","Алтайский край, Барнаул г",2015.0,...,1,1997-11-13,2015-01-01,1997,3,1,8,14,74,10
6,57717,16945,2013,Муж,СН,-,1995-05-10 00:00:00.000,-,-,NaN,...,1,1995-05-10,2013-01-01,1995,3,0,0,0,1,0
7,65208,18196,2014,Жен,СН,Английский язык,1996-05-20 00:00:00.000,"МБОУ ""Калманская СОШ""","Алтайский край, Калманский р-н, Калманка с",2014.0,...,1,1996-05-20,2014-01-01,1996,3,1,8,14,412,10
8,78608,19620,2016,Жен,СН,Английский язык,1994-09-05 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Россия, Алтайский край, г Барнаул",2016.0,...,1,1994-09-05,2016-01-01,1994,3,1,8,14,70,10
9,83857,19966,2017,Муж,ОО,Английский язык,1999-06-24 00:00:00.000,МБОУ Лицей №101,"Алтайский край, г. Барнаул",2017.0,...,1,1999-06-24,2017-01-01,1999,2,1,8,14,1310,10


In [64]:
scaler = MinMaxScaler()
input_train_scaled = scaler.fit_transform(input_train.loc[:, ['male', 'female', 'language', 'basis', 'country', 'region', 'city', 'parents_country']])
input_train_scaled[:, 7]

array([1., 1., 1., 1., 1., 1., 0., 1., 1., 1.])

In [65]:
list(columns_to_code.values())

['basis', 'language', 'country', 'region', 'city', 'parents_country']

In [66]:
columns_to_model = digit_columns + list(columns_to_code.values())
columns_to_model

['Код_группы',
 'Год_Поступления',
 'Пособие',
 'Общежитие',
 'Наличие_Матери',
 'Наличие_Отца',
 'Опекунство',
 'Село',
 'Иностранец',
 'КодФакультета',
 'СрБаллАттестата',
 'Статус',
 'male',
 'female',
 'birth_year_int',
 'basis',
 'language',
 'country',
 'region',
 'city',
 'parents_country']

In [67]:
input_train.loc[:, columns_to_model].to_csv('data/clean.csv', index_label='ind')

In [68]:
codes_rule = {}
for name, code in columns_to_code.items():
    target_dict = code_dict(name)
    codes_rule[name] = target_dict    
    clean_frame[columns_to_code[name]] = clean_frame[name].apply(get_code, args=(code_dict(name),))

clean_frame

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,sex_summ,birth_year,entrance,birth_year_int,basis,language,country,region,city,parents_country
0,72716,16019,2015,Жен,ОО,Английский язык,1998-02-08 00:00:00.000,"МБОУ ""СОШ №59""","Алтайский край, Барнаул г",2014.0,...,1,1998-02-08,2015-01-01,1998,2,1,8,14,74,10
1,63306,14895,2010,Муж,ЦН,-,1992-04-18 00:00:00.000,-,-,NaN,...,1,1992-04-18,2010-01-01,1992,4,0,8,14,74,10
2,82469,20103,2017,Жен,ДН,Английский язык,1999-03-21 00:00:00.000,МБОУ Алтайская СОШ №5,"Алтайский край, Алтайский р-н, Алтайское с",2017.0,...,1,1999-03-21,2017-01-01,1999,1,1,8,14,31,10
3,81661,19838,2017,Жен,БН,Английский язык,1994-12-08 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,1,1994-12-08,2017-01-01,1994,0,1,8,14,1310,10
4,81509,19803,2017,Жен,БН,Английский язык,1995-06-28 00:00:00.000,ФГБОУ ВО Алтайский государственный университет,"Алтайский край, г. Барнаул",2017.0,...,1,1995-06-28,2017-01-01,1995,0,1,8,14,1310,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13579,74283,17593,2016,Муж,ОО,Английский язык,1998-09-17 00:00:00.000,МБОУ СОШ №98,"Россия, Алтайский край, г Барнаул",2016.0,...,1,1998-09-17,2016-01-01,1998,2,1,8,14,70,10
13580,71516,19433,2015,Жен,ОО,Немецкий язык,1997-09-19 00:00:00.000,"МКОУ ""Георгиевская СОШ""","Алтайский край, Ребрихинский р-н, Георгиевка с",2015.0,...,1,1997-09-19,2015-01-01,1997,2,2,8,14,1310,10
13581,76970,18723,2016,Муж,СН,Английский язык,1998-03-20 00:00:00.000,МБОУ СОШ №113 им. Сергея Семенова,"Россия, Алтайский край, г Барнаул",2016.0,...,1,1998-03-20,2016-01-01,1998,3,1,8,14,70,10
13582,48548,14663,2012,Жен,ОО,-,1995-07-06 00:00:00.000,-,-,NaN,...,1,1995-07-06,2012-01-01,1995,2,0,8,0,1,10


In [69]:
frame_to_csv = clean_frame.loc[clean_frame['СрБаллАттестата'] < 100]

In [70]:

frame_to_csv.loc[:, columns_to_model].to_csv('data/clean_frame.csv', index_label='ind')

In [71]:
import yaml
with open('data/codes.yaml', 'w') as file:
    yaml.dump(codes_rule, file)